In [1]:
# dependencies

import pandas as pd
import sqlite3
from sqlalchemy import create_engine
from sqlalchemy import Column, Date, Integer, String
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import Session


In [2]:
# upload csvs
wine_file = "winemag-data_first150k.csv"
wine_i = "Wine_imports.csv"

wine_df = pd.read_csv(wine_file)
wine_import = pd.read_csv(wine_i)

wine_df.head(10)


,Unnamed: 0,country,description,designation,points,price,province,region_1,region_2,variety,winery
0,0,US,This tremendous 100% varietal wine hails from ...,Martha's Vineyard,96,235.0,California,Napa Valley,Napa,Cabernet Sauvignon,Heitz
1,1,Spain,"Ripe aromas of fig, blackberry and cassis are ...",Carodorum Selección Especial Reserva,96,110.0,Northern Spain,Toro,NaN,Tinta de Toro,Bodega Carmen Rodríguez
2,2,US,Mac Watson honors the memory of a wine once ma...,Special Selected Late Harvest,96,90.0,California,Knights Valley,Sonoma,Sauvignon Blanc,Macauley
3,3,US,"This spent 20 months in 30% new French oak, an...",Reserve,96,65.0,Oregon,Willamette Valley,Willamette Valley,Pinot Noir,Ponzi
4,4,France,"This is the top wine from La Bégude, named aft...",La Brûlade,95,66.0,Provence,Bandol,NaN,Provence red blend,Domaine de la Bégude
5,5,Spain,"Deep, dense and pure from the opening bell, th...",Numanthia,95,73.0,Northern Spain,Toro,NaN,Tinta de Toro,Numanthia
6,6,Spain,Slightly gritty black-fruit aromas include a s...,San Román,95,65.0,Northern Spain,Toro,NaN,Tinta de Toro,Maurodos
7,7,Spain,Lush cedary black-fruit aromas are luxe and of...,Carodorum Único Crianza,95,110.0,Northern Spain,Toro,NaN,Tinta de Toro,Bodega Carmen Rodríguez
8,8,US,This re-named vineyard was formerly bottled as...,Silice,95,65.0,Oregon,Chehalem Mountains,Willamette Valley,Pinot Noir,Bergström
9,9,US,The producer sources from two blocks of the vi...,Gap's Crown Vineyard,95,60.0,California,Sonoma Coast,Sonoma,Pinot Noir,Blue Farm


In [3]:
# switch out US for United States
replace = wine_df.replace({'country': 'US'}, {'country': 'United States'})

In [4]:
# rename column 0
rename_wine = replace.rename(columns={'Unnamed: 0': 'ID'})
rename_wine.head()

,ID,country,description,designation,points,price,province,region_1,region_2,variety,winery
0,0,United States,This tremendous 100% varietal wine hails from ...,Martha's Vineyard,96,235.0,California,Napa Valley,Napa,Cabernet Sauvignon,Heitz
1,1,Spain,"Ripe aromas of fig, blackberry and cassis are ...",Carodorum Selección Especial Reserva,96,110.0,Northern Spain,Toro,NaN,Tinta de Toro,Bodega Carmen Rodríguez
2,2,United States,Mac Watson honors the memory of a wine once ma...,Special Selected Late Harvest,96,90.0,California,Knights Valley,Sonoma,Sauvignon Blanc,Macauley
3,3,United States,"This spent 20 months in 30% new French oak, an...",Reserve,96,65.0,Oregon,Willamette Valley,Willamette Valley,Pinot Noir,Ponzi
4,4,France,"This is the top wine from La Bégude, named aft...",La Brûlade,95,66.0,Provence,Bandol,NaN,Provence red blend,Domaine de la Bégude


In [5]:
# print cleaner wine df
rename_wine.head(5)

,ID,country,description,designation,points,price,province,region_1,region_2,variety,winery
0,0,United States,This tremendous 100% varietal wine hails from ...,Martha's Vineyard,96,235.0,California,Napa Valley,Napa,Cabernet Sauvignon,Heitz
1,1,Spain,"Ripe aromas of fig, blackberry and cassis are ...",Carodorum Selección Especial Reserva,96,110.0,Northern Spain,Toro,NaN,Tinta de Toro,Bodega Carmen Rodríguez
2,2,United States,Mac Watson honors the memory of a wine once ma...,Special Selected Late Harvest,96,90.0,California,Knights Valley,Sonoma,Sauvignon Blanc,Macauley
3,3,United States,"This spent 20 months in 30% new French oak, an...",Reserve,96,65.0,Oregon,Willamette Valley,Willamette Valley,Pinot Noir,Ponzi
4,4,France,"This is the top wine from La Bégude, named aft...",La Brûlade,95,66.0,Provence,Bandol,NaN,Provence red blend,Domaine de la Bégude


In [6]:
# import csv print
wine_import.head(5)

,ID,Country,Imports,% change from 2018 imports
0,0,United States,"$6,487,295,000",0.60%
1,1,United Kingdom,"$4,429,517,000",1.30%
2,2,Germany,"$2,883,085,000",-9.10%
3,3,China,"$2,444,671,000",-14.40%
4,4,Canada,"$1,949,254,000",-2.40%


In [7]:
# print column names for wine
rename_wine.columns

Index(['ID', 'country', 'description', 'designation', 'points', 'price',
       'province', 'region_1', 'region_2', 'variety', 'winery'],
      dtype='object')

In [8]:
# print columns for import
wine_import.columns

Index(['ID', 'Country ', 'Imports', '% change from 2018 imports'], dtype='object')

In [9]:
# sqlite create engine

engine = create_engine('sqlite:///wine_db.sqlite', echo=False)

In [10]:
# create base
Base = declarative_base()

In [11]:
# all wine table create
class Wine(Base):
    __tablename__ = 'wine'
    id = Column(Integer, primary_key=True)
    country = Column(String)
    description = Column(String)
    designation = Column(String)
    points = Column(Integer)
    price = Column(Integer)
    province = Column(String)
    region_1 = Column(String)
    region_2 = Column(String)
    variety = Column(String)
    winery = Column(String)
    
    def __init__(self, country, description, designation, points, price, province, region01, region02, variety, winery):
        self.country = country
        self.description = description
        self.designation = designation
        self.points = points
        self.price = price
        self.province = province
        self.region_1 = region_1
        self.region_2 = region_2
        self.variety = variety
        self.winery = winery
        
    

In [13]:
# import table create
class Imports(Base):
    __tablename__= 'imports'
    id = Column(Integer, primary_key=True)
    country = Column(String)
    Imports_usd = Column(String)
    imports_change = Column(String)
    
    def __init__(self, country, imports_usd, imports_change):
        self.country = Country
        self.imports_usd = Imports_usd
        self.imports_change = imports_change
        


In [14]:
# set up session
session = Session (bind=engine)


In [15]:
# create all metadata base engine
Base.metadata.create_all(engine)

In [16]:
# create sqlite filepath wine_all
sqlite_path = "wine_db.sqlite"

engine = create_engine(f'sqlite:///{sqlite_path}')

In [17]:
# engine execute wine_all
wine_all_df = engine.execute("SELECT * FROM wine")

print(wine_all_df)

In [18]:
# engine execute imports

wine_import_df = engine.execute("SELECT * FROM imports")

print(wine_import_df)

In [14]:
# create table for wine
# engine.execute ('''CREATE TABLE wine (
# ID INT NOT NULL PRIMARY KEY,
# country VARCHAR,
# description VARCHAR,
# designation VARCHAR,
# points INTEGER,
# price INTEGER,
# province VARCHAR,
# region01 VARCHAR,
# region02 VARCHAR,
# variety VARCHAR,
# winery VARCHAR
# );''')

In [15]:
# create table for imports
# engine.execute('''CREATE TABLE imports (
# ID INT NOT NULL PRIMARY KEY,
# country VARCHAR(50) NOT NULL,
# imports_usd VARCHAR(50) NOT NULL,
# imports_change VARCHAR(50) NOT NULL
# );''')

In [43]:
# check table names
engine.table_names()

['import', 'imports', 'wine']

In [22]:
# create dataframe to pull into sqlite file
res = engine.execute('''Select * from wine;''')
pd.DataFrame(res)

,0,1,2,3,4,5,6,7,8,9,10
0,0,United States,This tremendous 100% varietal wine hails from ...,Martha's Vineyard,96,235.0,California,Napa Valley,Napa,Cabernet Sauvignon,Heitz
1,1,Spain,"Ripe aromas of fig, blackberry and cassis are ...",Carodorum Selección Especial Reserva,96,110.0,Northern Spain,Toro,None,Tinta de Toro,Bodega Carmen Rodríguez
2,2,United States,Mac Watson honors the memory of a wine once ma...,Special Selected Late Harvest,96,90.0,California,Knights Valley,Sonoma,Sauvignon Blanc,Macauley
3,3,United States,"This spent 20 months in 30% new French oak, an...",Reserve,96,65.0,Oregon,Willamette Valley,Willamette Valley,Pinot Noir,Ponzi
4,4,France,"This is the top wine from La Bégude, named aft...",La Brûlade,95,66.0,Provence,Bandol,None,Provence red blend,Domaine de la Bégude
...,...,...,...,...,...,...,...,...,...,...,...
150925,150925,Italy,Many people feel Fiano represents southern Ita...,None,91,20.0,Southern Italy,Fiano di Avellino,None,White Blend,Feudi di San Gregorio
150926,150926,France,"Offers an intriguing nose with ginger, lime an...",Cuvée Prestige,91,27.0,Champagne,Champagne,None,Champagne Blend,H.Germain
150927,150927,Italy,This classic example comes from a cru vineyard...,Terre di Dora,91,20.0,Southern Italy,Fiano di Avellino,None,White Blend,Terredora
150928,150928,France,"A perfect salmon shade, with scents of peaches...",Grand Brut Rosé,90,52.0,Champagne,Champagne,None,Champagne Blend,Gosset


In [21]:
# WINE turn pandas csv dataframe into sqlite database imports

rename_wine.to_sql(name='wine', con=engine, if_exists='append', index=False)

In [23]:
# replace columns names for wine
wine_all = replace.rename(columns={'0': 'ID', '1': 'Country', '2': 'Description', '3': 'Designation', '4': 'Points', '5': 'Price', '6': 'Province' , '7':'Region_1', '8': 'Region_2', '9': 'Variety', '10': 'Winery' })
wine_all.head()

,Unnamed: 0,country,description,designation,points,price,province,region_1,region_2,variety,winery
0,0,United States,This tremendous 100% varietal wine hails from ...,Martha's Vineyard,96,235.0,California,Napa Valley,Napa,Cabernet Sauvignon,Heitz
1,1,Spain,"Ripe aromas of fig, blackberry and cassis are ...",Carodorum Selección Especial Reserva,96,110.0,Northern Spain,Toro,NaN,Tinta de Toro,Bodega Carmen Rodríguez
2,2,United States,Mac Watson honors the memory of a wine once ma...,Special Selected Late Harvest,96,90.0,California,Knights Valley,Sonoma,Sauvignon Blanc,Macauley
3,3,United States,"This spent 20 months in 30% new French oak, an...",Reserve,96,65.0,Oregon,Willamette Valley,Willamette Valley,Pinot Noir,Ponzi
4,4,France,"This is the top wine from La Bégude, named aft...",La Brûlade,95,66.0,Provence,Bandol,NaN,Provence red blend,Domaine de la Bégude


In [24]:
# rename 0 to ID
fin_wine = wine_all.rename(columns={'Unnamed: 0': 'ID'})
fin_wine


,ID,country,description,designation,points,price,province,region_1,region_2,variety,winery
0,0,United States,This tremendous 100% varietal wine hails from ...,Martha's Vineyard,96,235.0,California,Napa Valley,Napa,Cabernet Sauvignon,Heitz
1,1,Spain,"Ripe aromas of fig, blackberry and cassis are ...",Carodorum Selección Especial Reserva,96,110.0,Northern Spain,Toro,NaN,Tinta de Toro,Bodega Carmen Rodríguez
2,2,United States,Mac Watson honors the memory of a wine once ma...,Special Selected Late Harvest,96,90.0,California,Knights Valley,Sonoma,Sauvignon Blanc,Macauley
3,3,United States,"This spent 20 months in 30% new French oak, an...",Reserve,96,65.0,Oregon,Willamette Valley,Willamette Valley,Pinot Noir,Ponzi
4,4,France,"This is the top wine from La Bégude, named aft...",La Brûlade,95,66.0,Provence,Bandol,NaN,Provence red blend,Domaine de la Bégude
...,...,...,...,...,...,...,...,...,...,...,...
150925,150925,Italy,Many people feel Fiano represents southern Ita...,NaN,91,20.0,Southern Italy,Fiano di Avellino,NaN,White Blend,Feudi di San Gregorio
150926,150926,France,"Offers an intriguing nose with ginger, lime an...",Cuvée Prestige,91,27.0,Champagne,Champagne,NaN,Champagne Blend,H.Germain
150927,150927,Italy,This classic example comes from a cru vineyard...,Terre di Dora,91,20.0,Southern Italy,Fiano di Avellino,NaN,White Blend,Terredora
150928,150928,France,"A perfect salmon shade, with scents of peaches...",Grand Brut Rosé,90,52.0,Champagne,Champagne,NaN,Champagne Blend,Gosset


In [39]:
# create sqlite file database for imports
res_two = engine.execute('''Select * from import;''')
pd.DataFrame(res_two)

,0,1,2,3
0,0,United States,"$6,487,295,000",0.60%
1,1,United Kingdom,"$4,429,517,000",1.30%
2,2,Germany,"$2,883,085,000",-9.10%
3,3,China,"$2,444,671,000",-14.40%
4,4,Canada,"$1,949,254,000",-2.40%
...,...,...,...,...
437,216,Kiribati,"$22,000",-29%
438,217,Somalia,"$10,000",-56.50%
439,218,Palestine,"$8,000",-84.30%
440,219,Pitcairn,"$6,000",0%


In [40]:
fin_imports = wine_import.rename(columns={'0': 'ID', '1': 'Country', '2': 'Imports', '3': '% change from 2018 imports'})
fin_imports


,ID,Country,Imports,% change from 2018 imports
0,0,United States,"$6,487,295,000",0.60%
1,1,United Kingdom,"$4,429,517,000",1.30%
2,2,Germany,"$2,883,085,000",-9.10%
3,3,China,"$2,444,671,000",-14.40%
4,4,Canada,"$1,949,254,000",-2.40%
...,...,...,...,...
216,216,Kiribati,"$22,000",-29%
217,217,Somalia,"$10,000",-56.50%
218,218,Palestine,"$8,000",-84.30%
219,219,Pitcairn,"$6,000",0%


In [38]:
# turn pandas csv dataframe into sqlite database imports

wine_import.to_sql(name='import', con=engine, if_exists='append', index=False)

In [41]:
# check that table import is coming through
pd.read_sql_query('select * from import', con=engine).head()

,ID,Country,Imports,% change from 2018 imports
0,0,United States,"$6,487,295,000",0.60%
1,1,United Kingdom,"$4,429,517,000",1.30%
2,2,Germany,"$2,883,085,000",-9.10%
3,3,China,"$2,444,671,000",-14.40%
4,4,Canada,"$1,949,254,000",-2.40%


In [42]:
# check that table wine is coming through
pd.read_sql_query('select * from wine', con=engine).head()

,id,country,description,designation,points,price,province,region_1,region_2,variety,winery
0,0,United States,This tremendous 100% varietal wine hails from ...,Martha's Vineyard,96,235.0,California,Napa Valley,Napa,Cabernet Sauvignon,Heitz
1,1,Spain,"Ripe aromas of fig, blackberry and cassis are ...",Carodorum Selección Especial Reserva,96,110.0,Northern Spain,Toro,None,Tinta de Toro,Bodega Carmen Rodríguez
2,2,United States,Mac Watson honors the memory of a wine once ma...,Special Selected Late Harvest,96,90.0,California,Knights Valley,Sonoma,Sauvignon Blanc,Macauley
3,3,United States,"This spent 20 months in 30% new French oak, an...",Reserve,96,65.0,Oregon,Willamette Valley,Willamette Valley,Pinot Noir,Ponzi
4,4,France,"This is the top wine from La Bégude, named aft...",La Brûlade,95,66.0,Provence,Bandol,None,Provence red blend,Domaine de la Bégude
